In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import linalg as alg

from sklearn.decomposition import FactorAnalysis, PCA, TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris

data = pd.read_excel (r'~/Dropbox/ROS/Arch_Plans_Paper/DataSheetsCleaned/Analysis_Of_catagorial_02.xlsx',sheet_name='Sheet1')


# Air - quality
df = pd.DataFrame(data, columns= ['Does the quality of air in this part of building have a negative effect on your work performance ?',
                                  'Do you have control over air condition system ?',
                                  'Do you feel sleepy or headache when you get to your work-space ?',
                                  'Health (especially cheast)',
                                  'Body activity',
                                  'How would you describe the ventilation and air quality of building  ',
                                  'Do you smell odor or unusual smell in your work-space ?'])


df.at[0, 'Body activity'] = 14
df.at[1, 'Body activity'] = 5
df.at[2, 'Body activity'] = 26


#df.dtypes
#df['CO2 at any moment (PPM)'] = pd.to_numeric(df['CO2 at any moment (PPM)'], errors='coerce') 
#df.dtypes
#df = df.dropna(inplace=True)
#df.dtypes
feature_names = df.iloc[0].T

#print (df)
df.head()

u, s, vh = alg.svd(df, full_matrices=False, compute_uv=True, hermitian=False)

TruncatedSVD(n_components=7, n_iter=11, random_state=42)
u.shape, s.shape, vh.shape

print('U=', np.size(u))
print('s=', s)

x = StandardScaler().fit_transform(df)
#print(x)
covariance_matrix = np.cov(x.T)
#new_covariance_matrix = covariance_matrix[~np.isnan(covariance_matrix)]

print(covariance_matrix)

pca = PCA(n_components=1)

pca.fit_transform(covariance_matrix)


eigen_values, eigen_vectors = alg.eig(covariance_matrix)
print("Eigenvector: \n",eigen_vectors,"\n")
print("Eigenvalues: \n", eigen_values, "\n")

#principalDf = pd.DataFrame(data = pca, columns = ['pca1', 'pca2'])
    
print(feature_names)

ax = plt.axes()

im = ax.imshow(np.corrcoef(x.T), cmap="RdBu_r", vmin=-1, vmax=1)
#featurenames = feature_names.index.tolist()
featurenames = ['Does the quality of air in this part of building have a negative effect on your work performance ?',
                'Do you have control over air condition system ?',
                'Do you feel sleepy or headache when you get to your work-space ?',
                'Health (especially cheast)',
                'Body activity ',
                'How would you describe the ventilation and air quality of building  ',
                'Do you smell odor or unusual smell in your work-space ?']
print(len(featurenames))

ax.set_xticks([0, 1, 2, 3, 4, 5, 6])
ax.set_xticklabels(featurenames , rotation=90)
ax.set_yticks([0, 1, 2, 3, 4, 5, 6])
ax.set_yticklabels(featurenames )
print(type(feature_names))
plt.colorbar(im).ax.set_ylabel("$r$", rotation=0)
ax.set_title("IEQ preference correlation matrix")
plt.tight_layout()

n_comps = 7

methods = [('PCA', PCA()),
           ('Unrotated FA', FactorAnalysis()),
           ('Varimax FA', FactorAnalysis(rotation='varimax'))]
fig, axes = plt.subplots(ncols=len(methods), figsize=(20, 16))

for ax, (method, fa) in zip(axes, methods):
    fa.set_params(n_components = n_comps)
    fa.fit(x)

    components = fa.components_.T
    print("\n\n %s :\n" % method)
    print(components)

    vmax = np.abs(components).max()
    ax.imshow(components, cmap="RdBu_r", vmax=vmax, vmin=-vmax)
    ax.set_yticks(np.arange(len(feature_names)))
    print(np.arange(len(feature_names)))
    if ax.is_first_col():
        ax.set_yticklabels(featurenames)
    else:
        ax.set_yticklabels([])
    ax.set_title(str(method))
    ax.set_xticks([0, 1, 2, 3, 4, 5, 6])
    ax.set_xticklabels(featurenames,rotation=90)
fig.suptitle("Factors")
plt.tight_layout()
plt.show()



LinAlgError: SVD did not converge